In [1]:
# We compare pairwise similarity performance on `spectra_file`
# We can use pickled version for much faster loading times
spectra_file = "GNPS-random-10k.mgf"

# We take a random sample of spectra from said file

# Minimum size:
chunk_sizes_min = 32

# Maximum size
chunk_sizes_max = 20_000

# how many points to evaluate (in logspace) between min and max
num_evals = 15

! nvidia-smi -L
! echo Number of CPU cores $(nproc)

GPU 0: NVIDIA H100 80GB HBM3 (UUID: GPU-82e22c7f-c6cf-939e-452a-865e05037f0e)
Number of CPU cores 224


In [2]:
import pickle
import sys
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
import matchms
from matchms.filtering import require_precursor_mz
from matchms.importing import load_from_mgf
from tqdm import tqdm
from simms.similarity import CudaCosineGreedy, CudaModifiedCosine
from simms.utils import Timer, download


np.random.seed(42)

raw_spectra = list(load_from_mgf(download(spectra_file)))

In [3]:
spectra = []
for s in raw_spectra:
    s = require_precursor_mz(s)
    if s is not None:
        spectra.append(s)

In [4]:
len(spectra)

9997

In [5]:
chunk_sizes = np.round(
    np.logspace(
        np.log2(chunk_sizes_min),
        np.log2(chunk_sizes_max),
        num=num_evals,
        base=2,
        endpoint=True,
    )
).astype(int)


def loop(chunk_size, kernel):
    references = np.random.choice(spectra, size=chunk_size)
    queries = np.random.choice(spectra, size=chunk_size)

    # Allow warm-up
    kernel = kernel(batch_size=4096)
    kernel.matrix(references[:4], queries[:4])
    with Timer() as timer:
        kernel.matrix(references, queries)
    return (
        kernel.__class__.__name__,
        timer.duration,
        len(references) * len(queries),
    )  # All samples


# We only have 1 GPU, so no parallel here
data = Parallel(1)(
    delayed(loop)(chunk_size, kernel)
    for chunk_size in tqdm(chunk_sizes)
    for kernel in [CudaCosineGreedy, CudaModifiedCosine]
)
data = pd.DataFrame(data, columns=["kernel", "time", "pairs"])
data["label"] = "H100SXM"

100%|██████████| 15/15 [02:23<00:00,  9.57s/it]


In [6]:
data.to_json(sys.stdout)

{"kernel":{"0":"CudaCosineGreedy","1":"CudaModifiedCosine","2":"CudaCosineGreedy","3":"CudaModifiedCosine","4":"CudaCosineGreedy","5":"CudaModifiedCosine","6":"CudaCosineGreedy","7":"CudaModifiedCosine","8":"CudaCosineGreedy","9":"CudaModifiedCosine","10":"CudaCosineGreedy","11":"CudaModifiedCosine","12":"CudaCosineGreedy","13":"CudaModifiedCosine","14":"CudaCosineGreedy","15":"CudaModifiedCosine","16":"CudaCosineGreedy","17":"CudaModifiedCosine","18":"CudaCosineGreedy","19":"CudaModifiedCosine","20":"CudaCosineGreedy","21":"CudaModifiedCosine","22":"CudaCosineGreedy","23":"CudaModifiedCosine","24":"CudaCosineGreedy","25":"CudaModifiedCosine","26":"CudaCosineGreedy","27":"CudaModifiedCosine","28":"CudaCosineGreedy","29":"CudaModifiedCosine"},"time":{"0":0.225115539,"1":0.3089680099,"2":0.2298015929,"3":0.3129809238,"4":0.2310005929,"5":0.3145695087,"6":0.246092089,"7":0.3252622313,"8":0.2404935337,"9":0.3393995343,"10":0.2415750679,"11":0.3423717562,"12":0.259517163,"13":0.3594537596,"